# Presentacion | Trabajo práctico final

Códigos | Aclaraciones


In [ ]:
!pip install spacy===3.3.0
!pip install nltk
!python -m spacy download es_core_news_sm
import re
#import contractions
from spacy.language import Language
import spacy
import numpy as np
#import joblib as jb
import  pandas as pd
from nltk.stem import WordNetLemmatizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.2 MB 4.7 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: spacy
    Found existing installation: spacy 3.3.1
    Uninstalling spacy-3.3.1:
      Successfully uninstalled spacy-3.3.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 681 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [ ]:
lemmatizer = WordNetLemmatizer()

@Language.component("personal_component")
def lower_case(doc) :

    doc=doc.text.lower().replace("\n","")  #Paso de objeto spacy.doc a str
    #doc=contractions.fix(doc)   esto es para las  contracciones en ingles
    doc=re.sub("((http|https):\/\/[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*[\w\-\@?^=%&amp;/~\+#])?)"," ",doc)
    doc=re.sub('[^a-z¡!áéíóú]', ' ', doc)

    a,b = 'áéíóúü','aeiouu'
    trans = str.maketrans(a,b)
    doc=doc.translate(trans)
    nlp.disable_pipe("personal_component")# Desabilitar esta tuveria es vital, porque si no el programa entra en un bucle infinito XD!!
    doc=nlp(doc)# Paso de str a un objeto spacy.doc
    return doc

nlp = spacy.load("es_core_news_sm",disable=["parser",'ner','morph'])

nlp.add_pipe("personal_component", name="personal_component", first=True)
nlp.enable_pipe("senter")

def preprocessing_text(texto):
    #documento = []
    confianza=0
    contador=0



    #nlp.enable_pipe("personal_component")




    review=texto
    review = nlp(review) #Al instanciar el objeto nlp deshabilito por defecto  el personal_component en pipeline
    nlp.enable_pipe("personal_component") #Lo que hago en esta linea es volverla a habilitarlo



    #review = [token.lemma_ for token in review if not token.is_stop | token.is_space ]
    review = [lemmatizer.lemmatize(token.text) for token in review if not token.is_stop | token.is_space ]

    review = [word for word in review if not len(word)<=1 or word == "!"]
    return review

In [ ]:
def preprocesing_df(df):
    preprocesado=[]
    for i in range(df.shape[0]):
        texto=preprocessing_text(df.iloc[i])
        preprocesado.append(texto)
    return preprocesado

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
#os.chdir("/content/drive/My Drive")
path="/content/drive/My Drive/tweets_tp_final/filtrado_argentinaUsuarios_Vacunas_Procesado.csv"



Mounted at /content/gdrive


In [ ]:
user_argentina = pd.read_csv(path ) # hay que lincarlo a los datos en drive

FileNotFoundError: ignored

In [ ]:
procesado=preprocesing_df(user_argentina.or_tw_text[:150000]) #para 150000 son 22 minutos

NameError: ignored

In [ ]:
def compara(indice):
    print("Texto sin procesar : ")
    print(user_argentina.or_tw_text.iloc[indice])
    print(" ")
    print("texto procesado : ")
    print(" ".join(procesado[indice]))

In [ ]:
compara(1556) # el numero va de 0 al 150000

Aanálisis de Sentimiento con **pysentimiento**

In [ ]:
!pip install pysentimiento
from pysentimiento import create_analyzer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 31.6 MB/s 
     |████████████████████████████████| 312 kB 63.9 MB/s 
     |████████████████████████████████| 175 kB 45.9 MB/s 
     |████████████████████████████████| 212 kB 55.4 MB/s 
     |████████████████████████████████| 101 kB 10.6 MB/s 
     |████████████████████████████████| 1.1 MB 39.0 MB/s 
     |████████████████████████████████| 140 kB 55.7 MB/s 
     |████████████████████████████████| 596 kB 56.7 MB/s 
     |████████████████████████████████| 127 kB 57.9 MB/s 
     |████████████████████████████████| 6.6 MB 38.5 MB/s 
     |████████████████████████████████| 94 kB 2.0 MB/s 
     |████████████████████████████████| 271 kB 38.4 MB/s 
     |████████████████████████████████| 144 kB 50.3 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=116f3a08325261c56402538ff04c5d3d88cbd43d8c395e25e7cb2803692b149e


In [ ]:
analyzer = create_analyzer(task="sentiment", lang="es")

Downloading:   0%|          | 0.00/334 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/838k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/925 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/415M [00:00<?, ?B/s]

In [ ]:
analyzer.predict("Hoy es un dia oscuro")

AnalyzerOutput(output=NEG, probas={NEG: 0.909, NEU: 0.089, POS: 0.002})

In [ ]:
enero_14_24=pd.read_csv("dataset_twitter/filtrado_argentinaUsuarios_Vacunas_Procesado_enero_18_24.csv") # hay que lincarlo con los datos

In [ ]:
enero_14_24=enero_14_24.sample(frac=1).reset_index(drop=True) #mezclo el dataset para relaizar una selección al azar

In [ ]:
def clasification_dataset(df):
    pos=[]
    neg=[]
    neu=[]
    for i in df.or_tw_text[:10000]: #selecciono 10000 tweets al azar
        diccionario=analyzer.predict(i).probas
        pos.append(diccionario["POS"])
        neg.append(diccionario["NEG"])
        neu.append(diccionario["NEU"])
    return pos,neg,neu


In [ ]:
pos,neg,neu=clasification_dataset(enero_14_24) # Esto en mi computadora personal tardo como 15 minutos para 10000 tweets

In [ ]:
short_df=pd.DataFrame({"Texto":enero_14_24.or_tw_text[:10000],"Positivo":pos,"Negativo":neg,"Neutro":neu}) #armo un nuevo dataset con los tweets ya clasificados

In [ ]:
short_df.to_csv("Clasification_enero_14_24_10000_muestras.csv") #Guardo el dataset ya clasificado

NameError: ignored

**WORD CLOUD**

In [ ]:
df=pd.read_csv("Clasification_confiltro_abril_5_11_30000_muestras") # La idea es cargar los 30000 textos ya clasificados

In [ ]:
positivos=df[df.Positivo>=0.75] # Aca claramente el criterio de filtro  es discutible , ustedes pueden probar con otros valor

In [ ]:
texto=positivos.Texto # Me quedo solo con el texto positivo/negativo


In [ ]:
texto.shape # con esto podemos saber cuantas filas de  texto quedan despues del filtro , si elijo un umbral muy alto puedo tener muy pocos textos .

In [ ]:
#  Aca Le robe el codigo a Tomas
# Definimos una función que recibe un string y devuelve el mismo string en minúscula y sin signos,
def clean_text_round1(text):
    # pasa las mayusculas del texto a minusculas
    text = text.lower()
    # reemplaza texto entre corchetes por un espacio en blanco.
    text = re.sub('\[.*?¿\]\%', ' ', text)
    # reemplaza singnos de puntuacion por un espacio en blanco.
    text = re.sub(f'[{re.escape(string.punctuation)}]', ' ', text)
    # remueve palabras que contienen numeros.
    text = re.sub('\w*\d\w*', '', text)    # \w hace las veces de cualquier caracter alfanumérico (en casi cualquier idioma) y el guión bajo
                                           # por eso este texto remueve cualquier conjunto de strings que contengan un número
    return text

# Defino una funcion anonima que al pasarle un argumento devuelve el resultado de aplicarle la funcion anterior a este mismo argumento
round1 = lambda x: clean_text_round1(x)

# Dataframe que resulta de aplicarle a las columnas la funcion de limpieza
data_clean = pd.DataFrame(positivos.Texto.apply(round1))

In [ ]:
list_data=list(data_clean.Texto) # Transformo a una lista de textos, solo cambio el tipo de dato , de dataframe  a lista

In [ ]:
import nltk
nltk.download('stopwords') # hay que descargar este modulo en particular para poder filtrar los stopwords.

lines = nltk.corpus.stopwords.words('spanish') # lines es una lista de stopwords

In [ ]:
from wordcloud import WordCloud # importo la funcion WordCloud
import matplotlib.pyplot as plt

# Creo el objeto WordCloud con determinados parametros y utilizando nuestra lista de stopwords
wc = WordCloud(stopwords=lines, background_color="white", colormap="Dark2",
               max_font_size=150, random_state=42)

In [ ]:
wc.generate("".join(list_data))  #Aca una sutileza el  join lo que hace es comprimir todas las filas de texto separados en un unico texto.
plt.imshow(wc, interpolation="bilinear")

In [ ]:
lines.append("https")  # Si en el wordcloud aparecen palabras sin sentido como por ejemplo https,t,co.Puden filtrarlo agregandolo a las stopwords y ejecutando de nuevo el objeto wc
lines.append("t")
lines.append("co")